In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '..')
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14, 9)
plt.rcParams['font.size'] = 16
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

import covid19

PALETTE = sns.color_palette()

## data

In [ ]:
istat_path = 'comuni_giornaliero.csv'
_, istat = covid19.data.read_istat(istat_path)
# _, istat_all = covid19.data.read_istat(istat_path, drop=False)
# istat = istat.sel(time=slice(None, '2020-06-30'))
istat

In [ ]:
istat.sel(year=2020).sum('location')

In [ ]:
mar_15_19_averages = istat.sel(time=slice('2020-03-01', '2020-04-30')).fillna(0).mean(['time', 'year'])
mar_15_19_averages.sum('location').values

In [ ]:
jan_feb_20_averages = istat.sel(year=2020, time=slice('2020-01-01', '2020-02-20')).fillna(0).mean('time')
jan_feb_20_averages.sum('location').values

In [ ]:
deaths_italy = istat.sel(year=2020).sum('location')

(deaths_italy.sel(time=slice('2020-02-21', '2020-04-30')) - jan_feb_20_averages.sum('location')).sum('time')

In [ ]:
deaths_italy_fraction = deaths_italy / jan_feb_20_averages.sum('location')

deaths_italy_fraction

## situation report

In [ ]:
ax = covid19.plot.plot_xarray(istat.sum(['location', 'age_class']), hue='year', xlim=(np.datetime64('2019-12-28'), np.datetime64('2020-06-30')))
_ = ax.legend()

In [ ]:
ax = covid19.plot.plot_xarray(deaths_italy.sel(time=slice('2020-01-01', '2020-02-20')), window=1, xlim=(np.datetime64('2019-12-28'), np.datetime64('2020-03-15')))
_ = ax.set(
    title='Decessi per tutte le cause (ISTAT)',
    ylabel='decessi'
)
_ = ax.legend(title='classi di età', loc='upper right')

In [ ]:
ax = covid19.plot.plot_xarray(deaths_italy, foreground_interval=(None, '2020-04-30'))
_ = ax.set(
    title='Decessi per tutte le cause (ISTAT)',
    ylabel='decessi'
)
_ = ax.legend(title='classi di età')

In [ ]:
ax = covid19.plot.plot_xarray(deaths_italy_fraction, window=5, foreground_interval=(None, '2020-04-28'))
# foreground_hue=['50-59', '60-69', '70-79', '80-89', '90+']
_ = ax.set(
    title='Indice dei decessi per classe di età (ISTAT)',
    ylabel='indice',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='classe di età')

In [ ]:
ax = covid19.plot.plot_xarray(deaths_italy_fraction, window=5, foreground_hue=['70-79'], foreground_interval=(None, '2020-04-28'))
# foreground_hue=['50-59', '60-69', '70-79', '80-89', '90+']
_ = ax.set(
    title='Indice dei decessi per classe di età (ISTAT)',
    ylabel='indice',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='classe di età')

In [ ]:
comune = "Torino"
FI = (None, '2020-04-28')
window = 3

istat_comune = istat.sel(year=2020, location=comune).fillna(0)
istat_comune_frazione = istat_comune / istat_comune.sel(time=slice('2020-01-01', '2020-02-20')).mean('time')

istat_comune_anni = istat.sel(location=comune, time=istat['time'] != np.datetime64('2020-02-29')).sum('age_class')
ax = covid19.plot.plot_xarray(istat_comune_anni, window=window, hue='year', foreground_interval=FI, foreground_hue=[2016, 2020])
_ = ax.set(
    title=f'Decessi per tutte le cause nel comune di {comune} (ISTAT) media mobile {window} giorni',
    ylabel='decessi',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='anno')

ax = covid19.plot.plot_xarray(istat_comune, window=window, foreground_interval=FI, foreground_hue=['70-79', '80-89', '90+'])
_ = ax.set(
    title=f'Decessi per tutte le cause nel comune di {comune} (ISTAT)',
    ylabel='decessi'
)
_ = ax.legend(title='classi di età')


ax = covid19.plot.plot_xarray(istat_comune_frazione, window=window, foreground_interval=FI, foreground_hue=['70-79', '80-89', '90+'])
_ = ax.set(
    title=f'Indice dei decessi per classe di età nel comune di {comune} (ISTAT)',
    ylabel='indice',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='classe di età')


In [ ]:
provincia = 'Ferrara'
FI = (None, '2020-04-30')
window = 7

istat_provincia = istat.groupby('province').sum().sel(province=provincia, year=2020)
istat_provincia_frazione = istat_provincia / istat_provincia.sel(time=slice('2020-01-01', '2020-02-20')).fillna(0).mean('time')

istat_provincia_anni = istat.sel(time=istat['time'] != np.datetime64('2020-02-29')).groupby('province').sum().sel(province=provincia).sum('age_class')
ax = covid19.plot.plot_xarray(istat_provincia_anni, window=window, hue='year', foreground_interval=FI, foreground_hue=[2016, 2020])
_ = ax.set(
    title=f'Decessi per tutte le cause nella provincia di {provincia} (ISTAT) media mobile {window} giorni',
    ylabel='decessi',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='anno')

ax = covid19.plot.plot_xarray(istat_provincia, window=window, foreground_interval=FI, foreground_hue=['70-79', '80-89', '90+'])
_ = ax.set(
    title=f'Decessi per tutte le cause nella provincia di {provincia} (ISTAT)',
    ylabel='decessi'
)
_ = ax.legend(title='classi di età')


ax = covid19.plot.plot_xarray(istat_provincia_frazione, window=window, foreground_interval=FI)
_ = ax.set(
    title=f'Indice dei decessi per classe di età nella provincia di {provincia} (ISTAT)',
    ylabel='indice',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='classe di età')


# istat_provincia_anni.sel(time='2020-03').sum('time')

In [ ]:
provincia = "Lodi"
FI = (None, '2020-04-30')
window = 3

istat_provincia = istat.groupby('province').sum().sel(province=provincia, year=2020)
istat_provincia_frazione = istat_provincia / istat_provincia.sel(time=slice('2020-01-01', '2020-02-20')).fillna(0).mean('time')

istat_provincia_anni = istat.sel(time=istat['time'] != np.datetime64('2020-02-29')).groupby('province').sum().sel(province=provincia).sum('age_class')
ax = covid19.plot.plot_xarray(istat_provincia_anni, window=window, hue='year', foreground_interval=FI, foreground_hue=[2016, 2020])
_ = ax.set(
    title=f'Decessi per tutte le cause nella provincia di {provincia} (ISTAT) media mobile {window} giorni',
    ylabel='decessi',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='anno')

ax = covid19.plot.plot_xarray(istat_provincia, window=window, foreground_interval=FI, foreground_hue=['90+'])
_ = ax.set(
    title=f'Decessi per tutte le cause nella provincia di {provincia} (ISTAT)',
    ylabel='decessi'
)
_ = ax.legend(title='classi di età')


ax = covid19.plot.plot_xarray(istat_provincia_frazione, window=window, foreground_interval=FI)
_ = ax.set(
    title=f'Indice dei decessi per classe di età nella provincia di {provincia} (ISTAT)',
    ylabel='indice',
)
_ = ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
_ = ax.legend(title='classe di età')


# istat_provincia_anni.sel(time='2020-03').sum('time')